## テーブル定義

In [16]:
import pandas as pd
# 法人テーブル
companies_df = pd.DataFrame({
   'corporate_number': pd.Series(dtype='str'),
   'name': pd.Series(dtype='str'),
   'kana': pd.Series(dtype='str'),
   'name_en': pd.Series(dtype='str'),
   'location': pd.Series(dtype='str'),
   'postal_code': pd.Series(dtype='str'),
   'status': pd.Series(dtype='str'),
   'fiscal_year': pd.Series(dtype='int64'),
   'update_date': pd.Series(dtype='str')
})


# 補助金テーブル
subsidies_df = pd.DataFrame({
    'id': pd.Series(dtype='int64'),
    'corporate_number': pd.Series(dtype='str'),
    'title': pd.Series(dtype='str'),
    'amount': pd.Series(dtype='str'),
    'date_of_approval': pd.Series(dtype='str'),
    'government_departments': pd.Series(dtype='str'),
    'subsidy_resource': pd.Series(dtype='str'),
    'target': pd.Series(dtype='str'),
    'note': pd.Series(dtype='str'),
    'fiscal_year': pd.Series(dtype='int64')
})


# a

In [ ]:
import pandas as pd
import aiohttp
import asyncio
import time

# API設定
# API_TOKEN = "YOUR_API_TOKEN"  # トークンを置き換えてください
API_TOKEN = "Hgt7Gj3Il3rQovE8OiU817jgmrKe9jnM"  # テスト後に削除
BASE_URL = "https://info.gbiz.go.jp/hojin"
HEADERS = {"X-hojinInfo-api-token": API_TOKEN}

# DataFrameテンプレート
companies_df = pd.DataFrame({
    'corporate_number': pd.Series(dtype='str'),
    'name': pd.Series(dtype='str'),
    'kana': pd.Series(dtype='str'),
    'name_en': pd.Series(dtype='str'),
    'location': pd.Series(dtype='str'),
    'postal_code': pd.Series(dtype='str'),
    'status': pd.Series(dtype='str'),
    'update_date': pd.Series(dtype='str')
})

subsidies_df = pd.DataFrame({
    'id': pd.Series(dtype='int64'),
    'corporate_number': pd.Series(dtype='str'),
    'title': pd.Series(dtype='str'),
    'amount': pd.Series(dtype='str'),
    'date_of_approval': pd.Series(dtype='str'),
    'government_departments': pd.Series(dtype='str'),
    'subsidy_resource': pd.Series(dtype='str'),
    'target': pd.Series(dtype='str'),
    'note': pd.Series(dtype='str')
})

processed_pages_df = pd.DataFrame({
    'data_type': pd.Series(dtype='str'),
    'page_number': pd.Series(dtype='int64')
})

async def fetch_subsidy_data(session, start_date, end_date, page=1):
    """非同期APIリクエスト - 補助金情報取得"""
    endpoint = f"{BASE_URL}/v1/hojin/updateInfo/subsidy"
    params = {
        'from': start_date,
        'to': end_date,
        'page': str(page)
    }
    
    try:
        async with session.get(endpoint, headers=HEADERS, params=params) as response:
            if response.status == 200:
                return await response.json()
            else:
                print(f"エラー: HTTPステータスコード {response.status}")
                return None
    except Exception as e:
        print(f"リクエストエラー: {e}")
        return None

def save_data_to_df(data):
    """DataFrameにデータを保存"""
    global companies_df, subsidies_df
    
    if not data or "hojin-infos" not in data:
        return 0, 0
    
    companies_count = 0
    subsidies_count = 0
    
    for company in data["hojin-infos"]:
        # 法人情報の保存
        new_company = pd.DataFrame({
            'corporate_number': [company.get("corporate_number", "")],
            'name': [company.get("name", "")],
            'kana': [company.get("kana", "")],
            'name_en': [company.get("name_en", "")],
            'location': [company.get("location", "")],
            'postal_code': [company.get("postal_code", "")],
            'status': [company.get("status", "")],
            'update_date': [company.get("update_date", "")]
        })
        
        companies_df = pd.concat([companies_df, new_company], ignore_index=True)
        companies_count += 1
        
        # 補助金情報の保存
        if "subsidy" in company and company["subsidy"]:
            for subsidy in company["subsidy"]:
                new_subsidy = pd.DataFrame({
                    'id': [len(subsidies_df) + 1],
                    'corporate_number': [company.get("corporate_number", "")],
                    'title': [subsidy.get("title", "")],
                    'amount': [subsidy.get("amount", "")],
                    'date_of_approval': [subsidy.get("date_of_approval", "")],
                    'government_departments': [subsidy.get("government_departments", "")],
                    'subsidy_resource': [subsidy.get("subsidy_resource", "")],
                    'target': [subsidy.get("target", "")],
                    'note': [subsidy.get("note", "")]
                })
                
                subsidies_df = pd.concat([subsidies_df, new_subsidy], ignore_index=True)
                subsidies_count += 1
    
    return companies_count, subsidies_count

def is_page_processed(page):
    """ページが処理済みかチェック"""
    global processed_pages_df
    return not processed_pages_df[
        (processed_pages_df['data_type'] == 'subsidy') & 
        (processed_pages_df['page_number'] == page)
    ].empty

def mark_page_processed(page):
    """ページを処理済みとしてマーク"""
    global processed_pages_df
    new_page = pd.DataFrame({
        'data_type': ['subsidy'],
        'page_number': [page]
    })
    processed_pages_df = pd.concat([processed_pages_df, new_page], ignore_index=True)

async def get_subsidy_data(start_date, end_date):
    """非同期実行で補助金情報を取得"""
    global companies_df, subsidies_df
    
    async with aiohttp.ClientSession() as session:
        # 最初に1ページ目を取得して総ページ数を確認
        initial_data = await fetch_subsidy_data(session, start_date, end_date, 1)
        if not initial_data or "totalPage" not in initial_data:
            print("総ページ数の取得に失敗しました")
            return
        
        total_pages = int(initial_data["totalPage"])
        print(f"補助金情報総ページ数: {total_pages}、期間: {start_date}～{end_date}")
        
        # 1ページ目の処理
        if not is_page_processed(1):
            companies, subsidies = save_data_to_df(initial_data)
            mark_page_processed(1)
            print(f"ページ 1: {companies}社の情報と{subsidies}件の補助金情報を保存")
        else:
            print("ページ 1は処理済みです")
        
        # 残りのページを10ページずつバッチ処理
        start_time = time.time()
        processed_pages = 1
        
        for start_page in range(2, total_pages + 1, 10):
            batch_tasks = []
            batch_pages = []
            
            for page in range(start_page, min(start_page + 10, total_pages + 1)):
                if not is_page_processed(page):
                    batch_tasks.append(fetch_subsidy_data(session, start_date, end_date, page))
                    batch_pages.append(page)
                else:
                    print(f"ページ {page} は処理済みです")
            
            if batch_tasks:
                print(f"バッチ取得: {len(batch_tasks)} ページ")
                results = await asyncio.gather(*batch_tasks)
                
                for i, result in enumerate(results):
                    page = batch_pages[i]
                    if result:
                        companies, subsidies = save_data_to_df(result)
                        mark_page_processed(page)
                        processed_pages += 1
                        
                        # 進捗表示
                        elapsed_time = time.time() - start_time
                        avg_time_per_page = elapsed_time / processed_pages if processed_pages > 0 else 0
                        remaining_pages = total_pages - processed_pages
                        est_remaining_time = avg_time_per_page * remaining_pages
                        
                        print(f"ページ {page}/{total_pages}: {companies}社, {subsidies}件 " +
                              f"(進捗: {processed_pages/total_pages*100:.1f}%, 残り約{est_remaining_time/60:.1f}分)")
                
                await asyncio.sleep(1)  # APIレート制限対策
    
    # 最終結果の表示
    elapsed_time = time.time() - start_time
    print(f"\n処理完了 ({elapsed_time/60:.1f}分)")
    print(f"データには{len(companies_df)}社の法人情報と{len(subsidies_df)}件の補助金情報があります")

def main():
    # データ取得期間を直接指定
    start_date = "20200401"
    end_date = "20210331"
    
    # データ取得実行
    asyncio.run(get_subsidy_data(start_date, end_date))
    
    # データセット保存
    companies_df.to_csv('companies_2020.csv', index=False)
    subsidies_df.to_csv('subsidies_2020.csv', index=False)
    
    # データ概要表示
    print("\n法人データ概要:")
    print(companies_df.info())
    
    print("\n補助金データ概要:")
    print(subsidies_df.info())

if __name__ == "__main__":
    main()